# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
filepath = 'data/song_data'

In [5]:
song_files = get_files(filepath)
#print(song_files)

In [6]:
df = pd.read_json(song_files[0], lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = (int(df.song_id[0]), str(df.title[0]), str(df.artist_id[0]), int(df.year[0]), float(df.duration[0]))
song_data

(1, "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = (str(df.artist_id[0]), str(df.artist_name[0]), str(df.artist_location[0]), df.artist_latitude[0], df.artist_longitude[0])
artist_data

('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
filepath = 'data/log_data'

In [12]:
log_files = get_files(filepath)
#print(log_files)

In [13]:
df = pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df["page"] == "NextSong"]
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
10,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1540266185796,9,Pump It,200,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
12,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Nobody Puts Baby In The Corner,200,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
13,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Mango Pickle Down River (With The Wilcannia Mob),200,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [15]:
t = df["ts"]
#t

In [16]:
datetimes_df = pd.to_datetime(df.ts, unit='ms')

#datetimes = str(list(datetimes_df))
#print(datetimes)

timestamps = list(t)
#print(timestamps)

hours = list(datetimes_df.dt.hour.values)
#print(hours)

days = list(datetimes_df.dt.day.values)
#print(days)

weeks = list(datetimes_df.dt.isocalendar().week.values)
#print(weeks)

months = list(datetimes_df.dt.month.values)
#print(months)

years = list(datetimes_df.dt.year.values)
#print(years)

weekdays = list(datetimes_df.dt.weekday.values)
#print(weekdays)


column_labels = ('datetime', 'hour', 'day', 'week', 'month', 'year', 'weekday')
#print(column_labels)

#print(np.transpose(datetimes.dt.hour.values))

#t["hour"] = datetimes.dt.hour.values
#t

#print(datetimes[2].to_pydatetime())

#time_data = (datetimes.values, datetimes.dt.hour.values, datetimes.dt.day.values, datetimes.dt.isocalendar().week.values, datetimes.dt.month.values,
#             datetimes.dt.year.values, datetimes.dt.weekday.values)


#column_labels = ()
#time_data

In [17]:
time_df = pd.DataFrame(list(zip(timestamps, hours, days, weeks, months, years, weekdays)), columns = column_labels)
time_df

,datetime,hour,day,week,month,year,weekday
0,1541106106796,21,1,44,11,2018,3
1,1541106352796,21,1,44,11,2018,3
2,1541106496796,21,1,44,11,2018,3
3,1541106673796,21,1,44,11,2018,3
4,1541107053796,21,1,44,11,2018,3
5,1541107493796,21,1,44,11,2018,3
6,1541107734796,21,1,44,11,2018,3
7,1541108520796,21,1,44,11,2018,3
8,1541109125796,21,1,44,11,2018,3
9,1541109325796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
13,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df.iterrows():

    print(index)
    print(row.song, row.artist, row.length)

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

2
You Gotta Be Des'ree 246.30812
None
4
Flat 55 Mr Oizo 144.03873
None
5
Quem Quiser Encontrar O Amor Tamba Trio 177.18812
None
6
Eriatarka The Mars Volta 380.42077
None
7
Becoming Insane Infected Mushroom 440.2673
None
8
Congratulations Blue October / Imogen Heap 241.3971
None
9
Once again Girl Talk 160.15628
None
10
Pump It Black Eyed Peas 214.93506
None
12
Nobody Puts Baby In The Corner Fall Out Boy 200.72444
None
13
Mango Pickle Down River (With The Wilcannia Mob) M.I.A. 233.7171
None
14
Eye Of The Tiger Survivor 245.36771
None


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.